In [4]:
import numpy as np
import xarray as xr

from xcube.core.store import new_data_store
from avl.verify import verify_dataset, ERROR

In [5]:
bucket_name = 'agriculture-vlab-data-staging'

In [16]:
store = new_data_store('s3', root=bucket_name, max_depth=5, storage_options=dict(anon=True))
# list(store.get_data_ids())

In [27]:

store = new_data_store('s3', 
                       root='agriculture-vlab-data-staging',
                       max_depth=5, 
                       storage_options=dict(anon=True))
ds = store.open_data("avl/l2a-s1/bel/S1_L2_COH_VV_31UFR.zarr", 
                     decode_cf=False)
np.all(ds.x.diff(dim='x') > 0), np.all(ds.y.diff(dim='y') < 0)

(<xarray.DataArray 'x' ()>
 array(False),
 <xarray.DataArray 'y' ()>
 array(False))

In [30]:
ds.x.values

array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [7]:
report = {}
for data_id in store.get_data_ids():
    try:
        # We use xarray to open the cube with 
        #store = s3fs.S3Map(f'{bucket_name}/{data_id}', s3=s3)
        #cube = xr.open_zarr(store, decode_cf=False) 
        dataset = store.open_data(data_id, decode_cf=False)
        issues = verify_dataset(dataset, level=ERROR)
        if not issues:
            print(f"Dataset {data_id}: OK")
        else:
            print(f"Dataset {data_id}: ERROR")
            report[data_id] = issues
    except Exception as e:
        print(f"Error for {data_id!r}: {e}")

report

Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UCQ.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UCR.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UDQ.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UDR.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UDS.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UDT.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UDU.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UEQ.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UER.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UES.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UET.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UEU.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UFQ.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UFR.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UFS.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UFT.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UFU.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UGQ.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BCK_VH_31UGR.zarr: OK
Dataset avl/l2a-s1/bel/S1_L2_BC

{'avl/l2a-s1/bel/S1_L2_COH_VH_31UER.zarr': [('ERROR',
   "values of variable 'time' must be monotonically increasing")],
 'avl/l2a-s1/bel/S1_L2_COH_VH_31UES.zarr': [('ERROR',
   "values of variable 'time' must be monotonically increasing")],
 'avl/l2a-s1/bel/S1_L2_COH_VH_31UET.zarr': [('ERROR',
   "values of variable 'time' must be monotonically increasing")],
 'avl/l2a-s1/bel/S1_L2_COH_VH_31UFR.zarr': [('ERROR',
   "values of variable 'time' must be monotonically increasing")],
 'avl/l2a-s1/bel/S1_L2_COH_VH_31UFS.zarr': [('ERROR',
   "values of variable 'time' must be monotonically increasing")],
 'avl/l2a-s1/bel/S1_L2_COH_VH_31UFT.zarr': [('ERROR',
   "values of variable 'time' must be monotonically increasing")],
 'avl/l2a-s1/bel/S1_L2_COH_VH_31UGR.zarr': [('ERROR',
   "values of variable 'time' must be monotonically increasing")],
 'avl/l2a-s1/bel/S1_L2_COH_VH_31UGS.zarr': [('ERROR',
   "values of variable 'time' must be monotonically increasing")],
 'avl/l2a-s1/bel/S1_L2_COH_VH_31

In [8]:
import json
with open("dataset-verification-details.json", "w") as fp:
    json.dump(report, fp, indent=2)

---



In [33]:
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
store = s3fs.S3Map('agriculture-vlab-data-staging/avl/l2a-s1/bel/S1_L2_COH_VV_31UFR.zarr', s3=s3)
ds = xr.open_zarr(store)